# Generates the raw figure shown in README.md

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from matplotlib import rc
plt.rc('legend', fontsize=18)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('axes', labelsize=22)
plt.rc('axes', titlesize=22)
from matplotlib import rcParams
rcParams['xtick.major.width'] = 2
rcParams['ytick.major.width'] = 2
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patheffects as pe

In [2]:
# Read digitized file

df = pd.read_feather("test_to_digitize_digitized.feather")

In [3]:
# Convert column, row, and BCID bins into x, y, and z coordinates

df['x'] = df['column'].apply(lambda x: x*250.)
df['y'] = df['row'].apply(lambda x: (335-x)*50.)
df['z'] = df['BCID'].apply(lambda x: x*216.)

In [4]:
# Shift digitized events to origin

df['x_shift'] = df['x'].apply(lambda x: x - x.min())
df['y_shift'] = df['y'].apply(lambda x: x - x.min())
df['z_shift'] = df['z'].apply(lambda x: x - x.min())

In [5]:
# Shift primary tracks to origin but give them an arbitrary offset to appear above digitized track

df['chipx_shift'] = (df['chipx'].apply(lambda x: x - x.min())*10000)+750
df['chipy_shift'] = (df['chipy'].apply(lambda x: x - x.min())*10000)+750
df['chipz_shift'] = (df['chipz'].apply(lambda x: x - x.min())*10000)+4000

In [6]:
%matplotlib qt

def plot_event(df, col1, col2, col3, col4, eventList, x_shift, y_shift, cmap = plt.cm.viridis, 
               cmap_shadow = plt.cm.gray, alpha = 1, alpha_shadow = 0.5, alpha_edge = 0, 
               alpha_shadow_edge = 0.07, cbar = True, save = False):
    def cuboid_data(o, size=(1,1,1)): #Faster cube generation than using voxels
        X = [[[0, 1, 0], [0, 0, 0], [1, 0, 0], [1, 1, 0]],
         [[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]],
         [[1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1]],
         [[0, 0, 1], [0, 0, 0], [0, 1, 0], [0, 1, 1]],
         [[0, 1, 0], [0, 1, 1], [1, 1, 1], [1, 1, 0]],
         [[0, 1, 1], [0, 0, 1], [1, 0, 1], [1, 1, 1]]]
        X = np.array(X).astype(float)
        for i in range(3):
            X[:,:,i] *= size[i]
        X += np.array(o)
        return X

    def plotCubeAt(positions,sizes=None,colors=None,edgecolors=None,edge_alpha=0, **kwargs): #generates collection of voxels
        if not isinstance(colors,(list,np.ndarray)): colors=["C0"]*len(positions)
        if not isinstance(sizes,(list,np.ndarray)): sizes=[(1,1,1)]*len(positions)
        if not isinstance(edgecolors,(list,np.ndarray)): edgecolors=[[0,0,0,.5]]*len(positions)
        g = []
        for p,s,c in zip(positions,sizes,colors):
            g.append( cuboid_data(p, size=s) )
        return Poly3DCollection(np.concatenate(g),  
                            facecolors=np.repeat(colors,6, axis=0), 
                                edgecolors=(0,0,0,edge_alpha), zorder = -1, **kwargs)
    
    def discretize_colormap(cmap):
        cmaplist = [cmap(i) for i in range(cmap.N)]
        # create the new map
        cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
            'Custom cmap', cmaplist, N = 14) # N = 14 for 14 unique TOT codes
        return cmap
    
    cmap = discretize_colormap(cmap)
    cmap_shadow = discretize_colormap(cmap_shadow)

    colors_dict = {}
    colors_dict_shadow = {}
    for i,color in enumerate(range(0,14)):
        colors_dict[color] = np.array(cmap(i))
        colors_dict[color][3] = alpha #sets alpha
        colors_dict_shadow[color] = np.array(cmap_shadow(i))
        colors_dict_shadow[color][3] = alpha_shadow #sets alpha
    
    fig = plt.figure(figsize = (18,14))
    #ax = fig.add_subplot(projection='3d', proj_type='ortho') #orthographic view
    ax = fig.add_subplot(projection='3d') #perspective view
    for i,event in enumerate(eventList):
        tmp = df.iloc[event] #filter out single event
        ax.scatter(tmp['chipx_shift']+x_shift[i],tmp['chipy_shift']+y_shift[i], 
                   tmp['chipz_shift'],color = 'k') #plot primary track
        positions = np.concatenate((tmp[col1][:, np.newaxis]+x_shift[i], 
                                    tmp[col2][:, np.newaxis]+y_shift[i], 
                                    tmp[col3][:, np.newaxis]), axis=1) #voxel positions
        positions2 = np.concatenate((tmp[col1][:, np.newaxis]+x_shift[i], 
                                     tmp[col2][:, np.newaxis]+y_shift[i], 
                                     np.array([0 for i in range(0,len(tmp[col3]))])[:, np.newaxis]), 
                                    axis=1) #shadow positions
        colors = pd.Series(tmp[col4].transpose()).map(colors_dict).to_numpy()
        colors_shadow = pd.Series(tmp[col4].transpose()).map(colors_dict_shadow).to_numpy()
        pc = plotCubeAt(positions, sizes=[(250, 50, 250)]*len(positions), colors=colors,edge_alpha = alpha_edge)
        ax.add_collection3d(pc)
        pc_shadow = plotCubeAt(positions2, sizes=[(250, 50, 0)]*len(positions), colors=colors_shadow,edge_alpha = alpha_shadow_edge)
        ax.add_collection3d(pc_shadow)
        
    ax.set_xlabel(r'x [mm]',labelpad = 16)
    ax.set_ylabel(r'y [mm]',labelpad = 18)
    ax.set_zlabel(r'z [mm]',labelpad = 11)
    ax.set_xlim(0,6000)
    ax.set_ylim(0,6000)
    ax.set_zlim(0,5000)
    ax.set_xticks([0,1000,2000,3000,4000,5000,6000])
    ax.set_yticks([0,1000,2000,3000,4000,5000,6000])
    ax.set_zticks([0,1000,2000,3000,4000,5000])
    ax.set_xticklabels([0,1,2,3,4,5,6])
    ax.set_yticklabels([0,1,2,3,4,5,6])
    ax.set_zticklabels([0,1,2,3,4,5])
    if cbar:
        bounds = np.linspace(-0.5, 13.5, 15)
        scat = ax.scatter([0 for i in range(0,14)], [0 for i in range(0,14)], c=[i for i in range(0,14)], s=0,
                  cmap=cmap)
        cbaxes = fig.add_axes([0.78, 0.37, 0.015, 0.29])
        co = fig.colorbar(scat,spacing='proportional',ticks=bounds+0.5, boundaries=bounds,shrink=0.5,cax = cbaxes)
        co.set_label(r'Q[TOT]',rotation=270,labelpad=25)
        print(co.ax.yaxis.get_ticklabels())
        for index, label in enumerate(co.ax.yaxis.get_ticklabels()):
            if index % 2 != 0:
                label.set_visible(False)
    #Adjust color palette of axes
    ax.zaxis.set_pane_color((0.8, 0.4, 0.4, 0.4))
    ax.xaxis.set_pane_color((1.0, 0.2, 0.4, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.xaxis._axinfo["grid"]['linewidth'] = 0.4
    ax.yaxis._axinfo["grid"]['linewidth'] =  0.4
    ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    #format axes
    ax.tick_params(axis='x', which='major', pad=3)
    ax.tick_params(axis='y', which='major', pad=7)
    ax.tick_params(axis='z', which='major', pad=5)
    tmp_planes = ax.zaxis._PLANES 
    ax.zaxis._PLANES = ( tmp_planes[2], tmp_planes[3], 
                     tmp_planes[0], tmp_planes[1], 
                     tmp_planes[4], tmp_planes[5])
    view_1 = (25, -135)
    view_2 = (25, -45)
    init_view = view_2
    ax.view_init(*init_view)
    ax.xaxis.set_minor_locator(MultipleLocator(250))
    ax.yaxis.set_minor_locator(MultipleLocator(250))
    ax.view_init(azim=224, elev=34) #
    fig.subplots_adjust(top = 0.95, bottom = 0.05, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    if save:
        plt.savefig('digi.png', dpi=300,bbox_inches='tight')

In [7]:
'''
col1, col2, col3, and col4 correspond to the x, y, z, and charge coordinates plotted
eventList = event indices to plot
x_shift, and y_shift both have the same length as eventList and are user-input arbitrary shifts for visualization
alpha = transparency of voxel shading
alpha_{edge,shadow,shadow_edge} = transparency of {voxel edge, voxel shadow fill, voxel shadow edge fill}
'''

plot_event(df, col1 = 'x_shift', col2='y_shift', col3 = 'z', col4 = 'tot', 
           eventList = np.array([0,1,2]), x_shift = [0,3600,300], y_shift = [0,0,3800], alpha = 1, 
           alpha_edge = 0.1, alpha_shadow = 0.07, alpha_shadow_edge = 0.03, cbar = True)

[Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, ''), Text(1, 0, '')]
